In [1]:
%cd /Users/trancatkhanh/Downloads/GAME
from base.Splendor_v3.env import *


/Users/trancatkhanh/Downloads/GAME


# Homework 3:
**1 agent tổng quán sử dụng bộ feature và bias có tỷ lệ thắng trên 60%**

# Bước 1 :Lấy bias của các action

In [2]:
from base.Splendor_v3.env import *
import numpy as np
bias1 = [np.random.rand(getActionSize()),np.zeros(getActionSize())]
@njit()
def p_bias(state,per):
    actions = getValidActions(state)
    actions *= per[0]
    action = np.argmax(actions)
    if getReward(state) == 1:
        per[1] += per[0]
    if getReward(state) == 0:
        per[0] = np.random.rand(getActionSize())
    return action,per

win, bias = numba_main_2(p_bias, 300000,bias1,0)
# Bias tối ưu 
bias_greedy = (bias[1]/np.max(bias[1]))
print(bias_greedy)
# Check tỷ lệ thắng của bias tối ưu 
@njit()
def p_bias_optim(state,per):
    actions = getValidActions(state)
    actions *= bias_greedy
    action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_optim, 1000,bias1,0)
print(win)

# Bước 2: Ghép các feature với nhau 

# Cách 1

In [ ]:
def get_feature5(old,new):
  #State old là state Hiện tại 
  #State new là state Mới 
  rate = 0 
  while rate<=0.999:

    # state mà có cả Value của old and new (1,0)
    o = (new>old)
    o= 1*o
    i = np.zeros((old.shape[0],1))
    for u in range(old.shape[0]) :
        if sum(o[u])>0 :
            # print(o[u])
            i[u] = 1
    #  a matrix with random value
    l = np.zeros((5,getStateSize()))
    for ii in range(0,5):
        rd1 = np.random.randint(getStateSize())
        rd2 = np.random.randint(getStateSize())
        while rd2 == rd1:
          rd2 = np.random.randint(getStateSize())
        rd3 = np.random.randint(getStateSize())
        while rd3 == rd2 or rd3 == rd1:
          rd3 = np.random.randint(getStateSize())
        l[ii][rd1] = 1
        l[ii][rd2] = 1
        l[ii][rd3] = -1
    l=l.reshape(161,5)
    #state mà kích hoạt (1,0)
    
    k = 1*((new@l)>=0)
    # state mà kích hoạt và có reward (1,0)
    y = np.zeros((old.shape[0],1))
    for p in range(old.shape[0]):
        if sum(k[p]) >= 1 and sum(i[p])>= 1:
            y[p] = 1
        else:
            y[p] = 0
      
    rate = (np.sum(y)/np.min(k))

  return l.reshape(5,161)

In [ ]:
win = 0
while win<60:
  matrix_feature_check_win_rate = []
  for k in range(0,3): 
    old = np.load('old'+f'{k}'+'.npy')
    new = np.load('new'+f'{k}'+'.npy')
    matrix  = get_feature5(old,new)
    matrix_feature = []
    matrix_feature.append(matrix)
    perx1=[np.zeros((5,getStateSize())),np.zeros((1,5))]
    for i in range(0,5):
        for j in range(getStateSize()):
            perx1[0][i][j] += matrix_feature[0][0][j]
    @njit
    def p0(state,per):
        actions = getValidActions(state)
        matran1 = per[0]@state    
        matran1 = 1*(matran1 >= 0)
        actions = np.where(actions == 1)[0]
        if np.sum(matran1) == 5 and k in actions :
            action = k
        else:
            list_action2 = getValidActions(state)
            actions2 = np.where(actions == 1)[0]
            action = actions2[np.random.randint(len(actions2))]
        return action,per
    win, x = numba_main_2(p0, 1000,perx1,0)
    while  win<350 :
      matrix  = get_feature5(old,new)
      matrix_feature = []
      matrix_feature.append(matrix)
      perx2=[np.zeros((5,getStateSize())),np.zeros((1,5))]
      for i in range(0,5):
          for j in range(getStateSize()):
              perx2[0][i][j] += matrix_feature[0][0][j]
      win, x = numba_main_2(p0, 1000,perx2,0)

    print('phần trăm trận thắng của action '+f'{k}',win/1000)

    matrix_feature_check_win_rate.append(matrix)
  perx=[np.zeros((15,getStateSize())),np.zeros((3,15))]

  for vk in range(0,3):
    for i in range(0,5):
        for j in range(getStateSize()):
            perx[0][vk*5+i][j] = matrix_feature_check_win_rate[vk][i][j]
            
  for i in range(0,3):
      for j in range(5):
          perx[1][i][i*5+j] = 1
  # print(perx[0][0:5])

  from base.Splendor_v3.env import *
  @njit
  def p_all_action(state,per):
      actions = getValidActions(state)
      matran1 = per[0]@state
      for i in range(len(matran1)):
          matran1[i] = int(matran1[i] >= 0)
      matran2 = per[1]@matran1
      matran2 *= actions
      if  np.sum(matran2) == 15 and np.max(matran2) ==5 :
          matran2 += bias_greedy
          action = np.argmax(matran2)
      elif np.sum(matran2) < 15 and np.max(matran2) ==5 and np.sum(matran2) >=10 :
          matran2 += bias_greedy
          action = np.argmax(matran2)
      else:        
          action = np.argmax(matran2)
      return action,per
  win, x = numba_main_2(p_all_action, 100,perx,0)
  print('số trận thắng để đạt 60%',win)
print('số trận thắng  60%',win)

phần trăm trận thắng của action 0 0.421
phần trăm trận thắng của action 1 0.472
phần trăm trận thắng của action 2 0.493
số trận thắng để đạt 60% 54
phần trăm trận thắng của action 0 0.432
phần trăm trận thắng của action 1 0.443
phần trăm trận thắng của action 2 0.48
số trận thắng để đạt 60% 57
phần trăm trận thắng của action 0 0.444
phần trăm trận thắng của action 1 0.46
phần trăm trận thắng của action 2 0.53
số trận thắng để đạt 60% 48
phần trăm trận thắng của action 0 0.391
phần trăm trận thắng của action 1 0.46
phần trăm trận thắng của action 2 0.504
số trận thắng để đạt 60% 43
phần trăm trận thắng của action 0 0.441
phần trăm trận thắng của action 1 0.48
phần trăm trận thắng của action 2 0.516
số trận thắng để đạt 60% 51
phần trăm trận thắng của action 0 0.43
phần trăm trận thắng của action 1 0.443
phần trăm trận thắng của action 2 0.513
số trận thắng để đạt 60% 44
phần trăm trận thắng của action 0 0.411
phần trăm trận thắng của action 1 0.434
phần trăm trận thắng của action 2 0.49

KeyboardInterrupt: 